# Convolutional Neural Network

### Importing the libraries

In [124]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [125]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [126]:
train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True
                                  rotation_range=20,           # Rotate the image by up to 20 degrees
    width_shift_range=0.2,       # Shift the image horizontally by up to 20% of the image width
    height_shift_range=0.2,      # Shift the image vertically by up to 20% of the image height
    shear_range=0.2,             # Shear the image by up to 20 degrees
    zoom_range=0.2,              # Zoom in or out on the image by up to 20%
    horizontal_flip=True,        # Flip the image horizontally
    fill_mode='nearest'
                                  
                                  )


training_set = train_datagen.flow_from_directory(r'trainset\rashes',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 20 images belonging to 3 classes.


### Preprocessing the Test set

In [127]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(r'testset\rashes',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 0 images belonging to 0 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [128]:
cnn = tf.keras.models.Sequential()



### Step 1 - Convolution

In [129]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [130]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [131]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [132]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [133]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [134]:
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [135]:
cnn.compile(loss = 'categorical_crossentropy', optimizer = 'adam', 
              metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [136]:
import os

dir_path = r'C:\Users\Geetha\Desktop\std_severity\trainset\rashes'

# Get a list of all the files in the directory
file_list = os.listdir(dir_path)

# Loop through the files and delete any with the ".ini" extension
for file_name in file_list:
    if file_name.endswith('.ini'):
        file_path = os.path.join(dir_path, file_name)
        os.remove(file_path)
#os.remove(r'C:\Users\Geetha\Desktop\STD_CNN\testset\muscle_wasting\desktop.ini')
#os.remove(r'C:\Users\Geetha\Desktop\STD_CNN\testset\yellowing_skin\desktop.ini')


In [137]:
import os
import cv2
bad_list=[]
dir=r'C:\Users\Geetha\Desktop\std_severity\trainset\rashes'
subdir_list=os.listdir(dir)
for d in subdir_list:  
    dpath=os.path.join (dir, d) 
    if d in ['testset', 'trainset']:
        class_list=os.listdir(dpath) 
       # print (class_list)
        for klass in class_list: 
            class_path=os.path.join(dpath, klass) 
            #print(class_path)
            file_list=os.listdir(class_path) 
            for f in file_list: 
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') 
                ext=f[index+1:] 
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    bad_list.append(fpath)                    
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        bad_list.append(fpath)
                       
print (bad_list)

[]


In [138]:
cnn.fit(x = training_set, validation_data=test_set , epochs = 25)


Epoch 1/25
1/1 [==============================] - 1s 1s/step - loss: 1.1010 - accuracy: 0.3000
Epoch 2/25
1/1 [==============================] - 1s 550ms/step - loss: 2.2134 - accuracy: 0.4500
Epoch 3/25
1/1 [==============================] - 0s 449ms/step - loss: 1.1732 - accuracy: 0.4500
Epoch 4/25
1/1 [==============================] - 1s 525ms/step - loss: 1.1357 - accuracy: 0.2500
Epoch 5/25
1/1 [==============================] - 1s 631ms/step - loss: 1.1396 - accuracy: 0.3500
Epoch 6/25
1/1 [==============================] - 1s 669ms/step - loss: 1.1215 - accuracy: 0.3000
Epoch 7/25
1/1 [==============================] - 1s 519ms/step - loss: 1.1105 - accuracy: 0.3000
Epoch 8/25
1/1 [==============================] - 1s 505ms/step - loss: 1.1008 - accuracy: 0.3000
Epoch 9/25
1/1 [==============================] - 1s 523ms/step - loss: 1.0904 - accuracy: 0.3000
Epoch 10/25
1/1 [==============================] - 0s 479ms/step - loss: 1.0729 - accuracy: 0.5000
Epoch 11/25
1/1 [=====

In [139]:

class_indices = training_set.class_indices
print(class_indices)

# Get the class label for a particular class




{'mild': 0, 'moderate': 1, 'severe': 2}


## Part 4 - Making a single prediction

In [140]:
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('single_prediction/severe_rash.jpg', target_size = (64, 64))
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result)
training_set.class_indices
if result[0][0] >result[0][1] and result[0][0] >result[0][2]:
  prediction = 'mild'
elif result[0][1] >result[0][0] and result[0][1] >result[0][2]:
  prediction = 'moderate'
elif result[0][2] >result[0][0] and result[0][2] >result[0][1]:
  prediction = 'severe'



probs = cnn.predict(test_image)
print(probs)

prob_sum = np.sum(probs)
print(prob_sum)

indices=np.argsort(probs)
print(indices)
print(probs[0][indices])
if probs[0][indices[0][0]]> probs[0][indices[0][1]]:
    print("a is larger than b")
else:
    print("b is larger than a")
    
top_3 = np.argsort(probs,axis=1)[:, -3:]
print(top_3)
class_probs = np.take_along_axis(probs, top_3, axis=1)
print(class_probs)

print(f"The test sample {test_image} ")
print("Top three Predicted classes and their corresponding probabilties\n")
for j in range(3):
        class_idx = top_3[0][2-j]
        class_prob = class_probs[0][2-j]
        print(f"\tClass {class_idx}: {class_prob:.5f}")
# print(f"Actual class to which the test sample belongs to {}  ")�


1/1 [==============================] - 0s 95ms/step
[[0. 0. 1.]]
1/1 [==============================] - 0s 37ms/step
[[0. 0. 1.]]
1.0
[[0 1 2]]
[[0. 0. 1.]]
b is larger than a
[[0 1 2]]
[[0. 0. 1.]]
The test sample [[[[ 96.  78.  64.]
   [112.  94.  72.]
   [119.  93.  70.]
   ...
   [121.  86.  58.]
   [149. 122.  95.]
   [160. 136. 108.]]

  [[ 93.  74.  57.]
   [101.  79.  56.]
   [115.  89.  66.]
   ...
   [182. 153. 121.]
   [128.  98.  72.]
   [147. 118.  88.]]

  [[ 79.  58.  41.]
   [ 91.  64.  45.]
   [106.  79.  58.]
   ...
   [179. 155. 119.]
   [105.  69.  43.]
   [129.  95.  70.]]

  ...

  [[105.  90.  67.]
   [105.  88.  68.]
   [101.  88.  69.]
   ...
   [ 92. 123. 128.]
   [ 96. 189. 222.]
   [103. 204. 224.]]

  [[102.  87.  68.]
   [106.  91.  70.]
   [105.  90.  69.]
   ...
   [ 95. 134. 141.]
   [ 94. 187. 220.]
   [102. 198. 223.]]

  [[103.  86.  68.]
   [106.  91.  70.]
   [106.  89.  73.]
   ...
   [ 96. 134. 145.]
   [ 95. 186. 217.]
   [114. 203. 233.]]]] 
To

In [141]:
print(prediction)

severe


In [142]:
print(result)

[[0. 0. 1.]]
